In [1]:
import pandas as pd
import os
import gensim.downloader
from gensim.models import Word2Vec, KeyedVectors
import tensorflow as tf
import numpy as np

from keras import Model, Input
from keras.layers import LSTM, Embedding, Dense
from keras.layers import InputLayer
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

#### Train word2vec model

In [2]:
def getSentences(path):
    file_path = path
    sentences = []
    current_sentence = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                if line.strip():
                    word = line.split()[0]
                    current_sentence.append(word)
                else:
                    if current_sentence:
                        sentences.append(current_sentence)
                    current_sentence = []
            if current_sentence:
                sentences.append(current_sentence)
            return sentences
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

def getTags(path):
    file_path = path
    tags = []
    current_sentence = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                if line.strip():
                    tag = line.split()[3]
                    current_sentence.append(tag)
                else:
                    if current_sentence:
                        tags.append(current_sentence)
                    current_sentence = []
            if current_sentence:
                tags.append(current_sentence)
            return tags
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [3]:
train_sentences = getSentences('./data/eng.train')
train_tags = getTags('./data/eng.train')

In [4]:
train_w2v = Word2Vec(train_sentences, vector_size=100, window=5, min_count=1, workers=4)

#### Load w2v Model

In [5]:
w2v = gensim.downloader.load("word2vec-google-news-300")

#### Preprocess Train, Development and Test Data

In [6]:
train_path = './data/eng.train'
development_path = './data/eng.testa'
test_path = './data/eng.testb'

In [7]:
def get_data(path):
    try:
        with open(path, 'r') as file:
            data = file.readlines()
        file.close()
    except Exception as e:
        data = None
        print(e)
    
    return data

In [8]:
train_data = get_data(train_path)
development_data = get_data(development_path)
test_data = get_data(test_path)

In [9]:
def extract_sentences(data):
    split_data = [line.split(' ') for line in data] if data != None else []
    sentences = []
    current_sentence = []
    for line in split_data:
        if line == ['\n']:
            sentences.append(current_sentence)
            current_sentence = []
        else:
            word = line[0]
            tag = line[-1].replace('\n', '')
            current_sentence.append([word, tag])
    sentences.append(current_sentence)
    return sentences

def extract_words_and_tags(data):
    words = []
    tags = []
    for line in data:
        parts = line.split()
        if parts:
            word = parts[0]
            tag = parts[-1]
            words.append(word)
            tags.append(tag)
    return words, tags

In [10]:
train_sentences = extract_sentences(train_data)
train_words, train_tags = extract_words_and_tags(train_data)
development_sentences = extract_sentences(development_data)
development_words, development_tags = extract_words_and_tags(development_data)
test_sentences = extract_sentences(test_data)
test_words, test_tags = extract_words_and_tags(test_data)

In [11]:
train_voc = np.unique(np.array(train_words))
dev_voc = np.unique(np.array(development_words))
tag_set = np.unique(np.array(train_tags))

In [12]:
tag_set

array(['B-LOC', 'B-MISC', 'B-ORG', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER',
       'O'], dtype='<U6')

In [13]:
print("Number of sentences (training):", len(train_sentences))
print("Number of sentences (dev):", len(development_sentences))
print("Number of sentences (test):", len(test_sentences))

Number of sentences (training): 14987
Number of sentences (dev): 3466
Number of sentences (test): 3684


In [14]:
print("Tag set (BIO):", tag_set)

Tag set (BIO): ['B-LOC' 'B-MISC' 'B-ORG' 'I-LOC' 'I-MISC' 'I-ORG' 'I-PER' 'O']


In [15]:
# train_pretrained_weights = train_w2v.wv.vectors
# train_num_tokens, train_embedding_dim = train_pretrained_weights.shape

# word2idx = train_w2v.wv.key_to_index
# word2idx['<UNK>'] = word2idx[list(word2idx.keys())[-1]]+1
# word2idx['<PAD>'] = word2idx[list(word2idx.keys())[-1]]+1

# voc = train_voc
# voc = np.append(voc,'<UNK>')
# voc = np.append(voc,'<PAD>')

# tag2idx = {k: v for v, k in enumerate(tag_set)}

In [16]:
train_pretrained_weights = w2v.vectors
train_num_tokens, train_embedding_dim = train_pretrained_weights.shape

word2idx = w2v.key_to_index
word2idx['<UNK>'] = word2idx[list(word2idx.keys())[-1]]+1
word2idx['<PAD>'] = word2idx[list(word2idx.keys())[-1]]+1
voc = word2idx.keys()
# voc = np.append(voc,'<UNK>')
# voc = np.append(voc,'<PAD>')

tag2idx = {k: v for v, k in enumerate(tag_set)}


In [17]:
word2idx['<UNK>']

3000000

In [18]:
num_tokens = len(voc) + 2
embedding_dim = 50
print(num_tokens)
for key in word2idx:
    if(word2idx[key] == 23629):
        print(key)

3000004
Tressel


In [19]:
def get_x_embeddings(sentences):
  sequence = []
  sent_seq = []
  for s in sentences:
    for w in s:
      if w[0] in word2idx.keys():
        sent_seq.append(word2idx[w[0]])
      else:
        sent_seq.append(word2idx['<UNK>'])
    sequence.append(sent_seq)
    sent_seq = []
  return sequence

In [20]:
x_train = get_x_embeddings(train_sentences)
y_train = [[tag2idx[w[1]] for w in s] for s in train_sentences]
x_dev = get_x_embeddings(development_sentences)
y_dev = [[tag2idx[w[1]] for w in s] for s in development_sentences]
x_test = get_x_embeddings(test_sentences)
y_test = [[tag2idx[w[1]] for w in s] for s in test_sentences]

x_train = pad_sequences(maxlen=embedding_dim, sequences=x_train, padding="post")
y_train = pad_sequences(maxlen=embedding_dim, sequences=y_train, padding="post", value=tag2idx['O'])
x_dev = pad_sequences(maxlen=embedding_dim, sequences=x_dev, padding="post")
y_dev = pad_sequences(maxlen=embedding_dim, sequences=y_dev, padding="post", value=tag2idx['O'])
x_test = pad_sequences(maxlen=embedding_dim, sequences=x_test, padding="post")
y_test = pad_sequences(maxlen=embedding_dim, sequences=y_test, padding="post", value=tag2idx['O'])

y_train = to_categorical(y_train)
y_dev = to_categorical(y_dev)
y_test = to_categorical(y_test)

In [27]:
x_train.shape
#y_train.shape

(14987, 50)

In [59]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import numpy as np
from tensorflow.keras.optimizers import Adam,SGD,Adagrad

num_epochs = 1000
batch_size = len(x_train)
num_classes = len(tag_set)
sequence_length = 50
output_shape=(sequence_length,num_classes)

model = Sequential([
    layers.Embedding(input_dim=num_tokens, output_dim=embedding_dim, input_length=sequence_length,trainable=False),
    layers.LSTM(sequence_length),
    layers.Dense(sequence_length*num_classes, activation='softmax'),
    layers.Reshape(output_shape)
    
])
desired_learning_rate = 0.001

# Create an Adam optimizer instance with the desired learning rate
optimizer = Adagrad(learning_rate=desired_learning_rate)

# Compile the model with the customized optimizer
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])
early_stopping = EarlyStopping(patience=10)
# Training (replace with your data)
model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, shuffle=True, validation_data=(x_dev, y_dev),
         callbacks = early_stopping, workers = 4)


Epoch 1/1000
1/1 [==============================] - 11s 11s/step - loss: 2.0795 - accuracy: 0.1267 - val_loss: 2.0794 - val_accuracy: 0.1269
Epoch 2/1000
1/1 [==============================] - 1s 837ms/step - loss: 2.0794 - accuracy: 0.1268 - val_loss: 2.0793 - val_accuracy: 0.1269
Epoch 3/1000
1/1 [==============================] - 1s 767ms/step - loss: 2.0793 - accuracy: 0.1268 - val_loss: 2.0793 - val_accuracy: 0.1270
Epoch 4/1000
1/1 [==============================] - 1s 1s/step - loss: 2.0793 - accuracy: 0.1269 - val_loss: 2.0792 - val_accuracy: 0.1271
Epoch 5/1000
1/1 [==============================] - 1s 948ms/step - loss: 2.0792 - accuracy: 0.1270 - val_loss: 2.0791 - val_accuracy: 0.1272
Epoch 6/1000
1/1 [==============================] - 1s 1s/step - loss: 2.0791 - accuracy: 0.1270 - val_loss: 2.0791 - val_accuracy: 0.1273
Epoch 7/1000
1/1 [==============================] - 1s 808ms/step - loss: 2.0791 - accuracy: 0.1271 - val_loss: 2.0790 - val_accuracy: 0.1273
Epoch 8/1000


In [22]:
num_classes = len(tag_set)
sequence_length = 50
output_shape=(sequence_length,num_classes)

model = keras.Sequential()
model.add(InputLayer(embedding_dim))
model.add(Embedding(input_dim=num_tokens, output_dim=embedding_dim, input_length = sequence_length, trainable=False,))
model.add(LSTM(units=sequence_length))
model.add(Dense(units=256, activation= "relu", kernel_regularizer=regularizers.L1L2(l1=0.025, l2=0.025)))
model.add(Dropout(0.01))  
model.add(Dense(sequence_length * num_classes, activation='softmax'))
model.add(tf.keras.layers.Reshape(output_shape))

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
early_stopping = EarlyStopping(patience=10)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            150000200 
                                                                 
 lstm (LSTM)                 (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 256)               13056     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 400)               102800    
                                                                 
 reshape (Reshape)           (None, 50, 8)             0         
                                                                 
Total params: 150136256 (572.72 MB)
Trainable params: 13

In [23]:
num_epochs = 1000
batch_size = 1000

model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, shuffle=True, validation_data=(x_dev, y_dev),
         callbacks = early_stopping, workers = 4)

Epoch 1/1000
15/15 [==============================] - 2s 93ms/step - loss: 24.1588 - accuracy: 0.7896 - val_loss: 21.2441 - val_accuracy: 0.9510
Epoch 2/1000
15/15 [==============================] - 1s 81ms/step - loss: 18.4655 - accuracy: 0.9546 - val_loss: 15.5912 - val_accuracy: 0.9510
Epoch 3/1000
15/15 [==============================] - 1s 82ms/step - loss: 13.6665 - accuracy: 0.9546 - val_loss: 11.6706 - val_accuracy: 0.9510
Epoch 4/1000
15/15 [==============================] - 1s 84ms/step - loss: 10.1054 - accuracy: 0.9546 - val_loss: 8.4623 - val_accuracy: 0.9510
Epoch 5/1000
15/15 [==============================] - 1s 86ms/step - loss: 7.1883 - accuracy: 0.9546 - val_loss: 5.8641 - val_accuracy: 0.9510
Epoch 6/1000
15/15 [==============================] - 1s 82ms/step - loss: 4.8466 - accuracy: 0.9546 - val_loss: 3.8075 - val_accuracy: 0.9510
Epoch 7/1000
15/15 [==============================] - 1s 83ms/step - loss: 3.0277 - accuracy: 0.9546 - val_loss: 2.2583 - val_accuracy:

In [60]:
predicted_labels = model.predict(x_test)
print(predicted_labels)

116/116 [==============================] - 1s 3ms/step
[[[0.00248762 0.00249715 0.00250286 ... 0.00247351 0.00251192 0.00250344]
  [0.00248729 0.00242283 0.00254964 ... 0.00257035 0.00241176 0.00261857]
  [0.00244001 0.00244937 0.00248798 ... 0.00251576 0.00254213 0.00260847]
  ...
  [0.00240743 0.00249032 0.0025015  ... 0.00244294 0.0024741  0.00259476]
  [0.00248156 0.00248135 0.00248282 ... 0.00249598 0.00242559 0.00260567]
  [0.00243995 0.00244739 0.00246903 ... 0.00249373 0.00245333 0.00253477]]

 [[0.00248762 0.00249721 0.00250293 ... 0.00247345 0.00251196 0.00250347]
  [0.00248728 0.00242285 0.00254963 ... 0.0025704  0.00241182 0.0026186 ]
  [0.00243995 0.00244939 0.002488   ... 0.00251579 0.0025421  0.00260843]
  ...
  [0.00240737 0.00249035 0.0025015  ... 0.00244294 0.00247408 0.00259482]
  [0.00248148 0.0024814  0.00248281 ... 0.00249594 0.0024255  0.00260568]
  [0.00243994 0.00244746 0.00246896 ... 0.0024938  0.00245331 0.00253478]]

 [[0.00248762 0.0024972  0.00250292 ... 0

In [61]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

116/116 [==============================] - 0s 2ms/step - loss: 2.0298 - accuracy: 0.8910
Test Loss: 2.029787302017212, Test Accuracy: 0.8909717798233032
